In [ ]:
# ===============================================================================
# 8. VISUALIZAÇÕES DE PERFORMANCE
# ===============================================================================

def plot_model_performance(results, predictions, y_test):
    """Visualiza performance dos modelos"""
    
    print("\n📈 VISUALIZAÇÕES DE PERFORMANCE")
    print("="*40)
    
    # 1. Comparação de métricas
    models_names = list(results.keys())
    r2_scores = [results[model]['Test R²'] for model in models_names]
    rmse_scores = [results[model]['Test RMSE'] for model in models_names]
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('R² Score Comparison', 'RMSE Comparison', 
                       'Predictions vs Actual', 'Residuals Analysis')
    )
    
    # R² comparison
    fig.add_trace(
        go.Bar(x=models_names, y=r2_scores, name='R² Score',
               marker_color='lightblue'),
        row=1, col=1
    )
    
    # RMSE comparison
    fig.add_trace(
        go.Bar(x=models_names, y=rmse_scores, name='RMSE',
               marker_color='lightcoral'),
        row=1, col=2
    )
    
    # Best model predictions vs actual
    best_model_name = max(results.keys(), key=lambda k: results[k]['Test R²'])
    best_predictions = predictions[best_model_name]
    
    fig.add_trace(
        go.Scatter(x=y_test, y=best_predictions, mode='markers',
                  name=f'{best_model_name} Predictions',
                  marker=dict(size=8, color='green')),
        row=2, col=1
    )
    
    # Perfect prediction line
    min_val = min(min(y_test), min(best_predictions))
    max_val = max(max(y_test), max(best_predictions))
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                  mode='lines', name='Perfect Prediction',
                  line=dict(color='red', dash='dash')),
        row=2, col=1
    )
    
    # Residuals
    residuals = y_test - best_predictions
    fig.add_trace(
        go.Scatter(x=best_predictions, y=residuals, mode='markers',
                  name='Residuals', marker=dict(size=8, color='purple')),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(x=[min(best_predictions), max(best_predictions)], y=[0, 0],
                  mode='lines', name='Zero Line',
                  line=dict(color='black', dash='dash')),
        row=2, col=2
    )
    
    fig.update_layout(height=800, showlegend=True,
                     title_text=f"Análise de Performance - Melhor Modelo: {best_model_name}")
    fig.show()
    
    return best_model_name

# ===============================================================================
# 9. INSIGHTS E RECOMENDAÇÕES AGRONÔMICAS
# ===============================================================================

def generate_agricultural_insights(df, importance_df, correlations):
    """Gera insights agronômicos baseados na análise"""
    
    print("\n🌾 INSIGHTS AGRONÔMICOS")
    print("="*40)
    
    insights = []
    
    # 1. Análise de Nutrientes
    nitrogen_corr = correlations.get('Nitrogen', 0)
    potash_corr = correlations.get('POTASH', 0)
    phosphate_corr = correlations.get('PHOSPHATE', 0)
    
    if nitrogen_corr > 0.5:
        insights.append("💡 NITROGÊNIO: Forte correlação positiva com produção. Aumentar aplicação de N pode incrementar rendimento.")
    elif nitrogen_corr < -0.3:
        insights.append("⚠️  NITROGÊNIO: Correlação negativa detectada. Possível excesso causando perdas.")
    
    # 2. Análise Hídrica
    rain_corr = correlations.get('avg_rain', 0)
    annual_corr = correlations.get('ANNUAL', 0)
    
    if rain_corr > 0.3:
        insights.append("💧 IRRIGAÇÃO: Chuva média mostra correlação positiva. Manejo hídrico é crucial.")
    
    # 3. Análise de Solo
    soil_diversity_mean = df['Soil_Diversity'].mean()
    insights.append(f"🏞️  SOLO: Diversidade média de tipos de solo: {soil_diversity_mean:.1f}")
    
    # 4. Eficiência
    water_eff_mean = df['Water_Efficiency'].mean()
    insights.append(f"⚡ EFICIÊNCIA HÍDRICA: Média de {water_eff_mean:.2f} kg/mm de chuva")
    
    # 5. Recomendações baseadas em ranges ótimos
    high_producers = df[df['RICE_PRODUCTION'] > df['RICE_PRODUCTION'].quantile(0.75)]
    
    optimal_nitrogen = high_producers['Nitrogen'].mean()
    optimal_potash = high_producers['POTASH'].mean()
    optimal_phosphate = high_producers['PHOSPHATE'].mean()
    optimal_rain = high_producers['avg_rain'].mean()
    
    insights.extend([
        f"🎯 FÓRMULA ÓTIMA (baseada nos 25% mais produtivos):",
        f"   • Nitrogênio: {optimal_nitrogen:,.0f} kg/ha",
        f"   • Potássio: {optimal_potash:,.0f} kg/ha", 
        f"   • Fósforo: {optimal_phosphate:,.0f} kg/ha",
        f"   • Chuva ideal: {optimal_rain:.1f} mm/mês"
    ])
    
    # 6. Alertas de Sustentabilidade
    max_nitrogen = df['Nitrogen'].max()
    if max_nitrogen > 100000:  # Threshold alto
        insights.append("⚠️  SUSTENTABILIDADE: Níveis muito altos de N detectados. Revisar para evitar lixiviação.")
    
    print("\n".join(insights))
    
    return insights, {
        'optimal_nitrogen': optimal_nitrogen,
        'optimal_potash': optimal_potash,
        'optimal_phosphate': optimal_phosphate,
        'optimal_rain': optimal_rain
    }

# ===============================================================================
# 10. FUNÇÃO PRINCIPAL
# ===============================================================================

def main():
    """Função principal que executa toda a análise"""
    
    print("🚀 INICIANDO ANÁLISE COMPLETA DE PRODUÇÃO DE ARROZ")
    print("="*60)
    
    # 1. Carregar dados
    df = load_and_explore_data('X1.csv')
    if df is None:
        return
    
    # 2. Feature Engineering
    df_enhanced = feature_engineering(df)
    
    # 3. EDA
    correlations = perform_eda(df_enhanced)
    
    # 4. Detecção de outliers
    outliers = detect_outliers(df_enhanced)
    
    # 5. Preprocessamento
    X_train, X_test, y_train, y_test, scaler = preprocess_data(df_enhanced)
    
    # 6. Modelagem
    results, predictions, results_df = train_models(X_train, X_test, y_train, y_test)
    
    # 7. Melhor modelo
    best_model_name = max(results.keys(), key=lambda k: results[k]['Test R²'])
    best_model = results[best_model_name]['Model']
    
    print(f"\n🏆 MELHOR MODELO: {best_model_name}")
    print(f"   R² Score: {results[best_model_name]['Test R²']:.4f}")
    print(f"   RMSE: {results[best_model_name]['Test RMSE']:.2f}")
    
    # 8. Interpretabilidade
    importance_df = interpret_model(best_model, X_train, X_test, X_train.columns)
    
    # 9. Visualizações de performance
    plot_model_performance(results, predictions, y_test)
    
    # 10. Insights agronômicos
    insights, optimal_params = generate_agricultural_insights(df_enhanced, importance_df, correlations)
    
    # 11. Resumo executivo
    print("\n" + "="*60)
    print("📊 RESUMO EXECUTIVO")
    print("="*60)
    print(f"🎯 Melhor Modelo: {best_model_name}")
    print(f"📈 Acurácia (R²): {results[best_model_name]['Test R²']:.1%}")
    print(f"📉 Erro Médio: ±{results[best_model_name]['Test RMSE']:.0f} kg/ha")
    print(f"🌾 Dataset: {len(df)} observações, {len(X_train.columns)} features")
    
    if importance_df is not None:
        top_feature = importance_df.iloc[0]['Feature']
        print(f"🔑 Feature mais importante: {top_feature}")
    
    print(f"💧 Eficiência hídrica média: {df_enhanced['Water_Efficiency'].mean():.2f} kg/mm")
    print("✅ Análise completa finalizada!")
    
    return {
        'df': df_enhanced,
        'results': results,
        'best_model': best_model,
        'best_model_name': best_model_name,
        'importance': importance_df,
        'insights': insights,
        'optimal_params': optimal_params
    }

# ===============================================================================
# EXECUTAR ANÁLISE
# ===============================================================================

if __name__ == "__main__":
    # Executar análise completa
    analysis_results = main()
    
    # Salvar resultados
    print("\n💾 Salvando resultados...")
    
    # Salvar modelo (exemplo com pickle)
    # import pickle
    # with open('best_rice_model.pkl', 'wb') as f:
    #     pickle.dump(analysis_results['best_model'], f)
    
    # Salvar insights
    # with open('agricultural_insights.txt', 'w') as f:
    #     f.write('\n'.join(analysis_results['insights']))
    
    print("🎉 Projeto concluído com sucesso!")
    print("📁 Arquivos prontos para portfólio!")